# Final Project NLP Smart Scientific Content Auto-completion System


a text auto-complete system that supports scientists in writing articles in:
- completing some typed words/letters.
- Identifying words/phrases that are very unlikely to occur in some text contexts and suggesting proper replacements.


Karmel Salah
15/4/2022

# Importing Libraries

In [1]:
!pip install pyenchant

/usr/lib/python3/dist-packages/secretstorage/dhcrypto.py:15: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/usr/lib/python3/dist-packages/secretstorage/util.py:19: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
Defaulting to user installation because normal site-packages is not writeable


In [2]:
import pandas as pd
import numpy as np
import os, glob
import math
import matplotlib.pyplot as plt
import seaborn as sns
#for text pre-processing
import re, string
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import SnowballStemmer
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer
from keras.utils import np_utils
from sklearn.model_selection import train_test_split
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
import enchant

nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
nltk.download('omw-1.4')
#for model-building
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Embedding, LSTM, Dense, Bidirectional
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import classification_report, f1_score, accuracy_score, confusion_matrix
from sklearn.metrics import roc_curve, auc, roc_auc_score
# bag of words
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
#for word embedding
import gensim
from gensim.models import Word2Vec

2022-05-09 22:10:22.237396: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-05-09 22:10:22.237429: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
/usr/lib/python3/dist-packages/requests/__init__.py:89: RequestsDependencyWarning: urllib3 (1.26.8) or chardet (3.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "
[nltk_data] Downloading package punkt to /home/karmel/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/karmel/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to /home/karmel/nltk_data...
[n

In [3]:
# get the names of each folders
files_names = os.listdir('dataset/')
len(files_names)

1064

# Convert to DataFrame

In [4]:
df = pd.DataFrame([], columns = ["id", "articles"])
print(f"data frame is {df}")

data frame is Empty DataFrame
Columns: [id, articles]
Index: []


In [5]:
def convert_to_df(df, files_names):
        id = []
        articles= []
        # Define files place and read each sub file for cleaning
        for index, file in enumerate(files_names):
                # read each article files
                path = os.getcwd()
                file_path = '{}/dataset/{}'.format(path,file)
                with open(file_path,'r', encoding='utf-16',  errors="ignore") as f:
                    texts = f.read() 
                    articles.append(texts) 
                    id.append(index)
                    f.close()
        print("the ids", id)
        print("the articles", articles)
        df['id'] = id
        df["articles"] = articles
        return df


In [6]:
df = convert_to_df(df, files_names)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [7]:
df["articles"]

0       Statistical Machine Translation\n\nDraft of Ch...
1       Found in Translation:\n\nLearning Robust Joint...
2       Available online at www.sciencedirect.com\n\nS...
3       J OURNAL OF I NFORMATION S CIENCE AND E NGINEE...
4       The University of Sheﬃeld\n\nT. E. Dunning\n\n...
                              ...                        
1059    Multi-label Hate Speech and Abusive Language D...
1060    hULMonA (      ): The Universal Language Model...
1061    Mazajak: An Online Arabic Sentiment Analyser\n...
1062    2017 2nd International Conferences on Informat...
1063    INVESTIGATIVE RADIOLOGY Volume 28, Number 6, 4...
Name: articles, Length: 1064, dtype: object

In [8]:
# count the number of words for each doc
df['word_count'] = df['articles'].apply(lambda x: len(str(x).split()))
df["word_count"]

0       48052
1        7759
2        4223
3        8574
4       71427
        ...  
1059     8225
1060     6109
1061     4263
1062     2044
1063     5910
Name: word_count, Length: 1064, dtype: int64

In [9]:
# total number of words of all articles  before processing
count = [x for x in df['word_count']]

sum = 0
for i in count:
    sum = sum + i
sum

8323220

 # Text pre-processing:

In [10]:
#convert to lowercase, strip and remove punctuations
def preprocess(text):
    text = text.lower()
    # Remove leading and trailing spaces
    text= text.strip()  
    text= re.compile('<.*?>').sub('', text) 
    text = re.compile('[%s]' % re.escape(string.punctuation)).sub(' ', text)  
    text = re.sub('\s+', ' ', text)  
    text = re.sub(r'\[[0-9]*\]',' ',text) 
    text= re.sub(r'[^\w\s]', '', str(text).lower().strip())
    text = re.sub(r'\d',' ',text) 
    text = re.sub(r'\s+',' ',text)
    # get ride of one letter word!
    text = re.sub(r'\b\w\b','',text) 
    
      
    return text

In [11]:
# STOPWORD REMOVAL
def stopword(string):
    a= [i for i in string.split() if i not in stopwords.words('english')]
    return ' '.join(a)

In [12]:
#LEMMATIZATION
# Initialize the lemmatizer
wl = WordNetLemmatizer()
 
# This is a helper function to map NTLK position tags
def get_wordnet_pos(tag):
    if tag.startswith('J'):
        return wordnet.ADJ
    elif tag.startswith('V'):
        return wordnet.VERB
    elif tag.startswith('N'):
        return wordnet.NOUN
    elif tag.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN
    
# Tokenize the sentence
def lemmatizer(string):
    word_pos_tags = nltk.pos_tag(word_tokenize(string)) # Get position tags
    a=[wl.lemmatize(tag[0], get_wordnet_pos(tag[1])) for idx, tag in enumerate(word_pos_tags)] # Map the position tag and lemmatize the word/token
    return " ".join(a)

In [13]:
#STEMMING
# Initialize the stemming
sb = SnowballStemmer("english")

#Tokenize the text
def stemming(stripped):
    word_pos_tags = nltk.pos_tag(word_tokenize(string)) # Get position tags
    a=[sb.stem(tag[0], get_wordnet_pos(tag[1])) for idx, tag in enumerate(word_pos_tags)] # Map the position tag and lemmatize the word/token
    return " ".join(a)

In [14]:
#Remove non English words
dictionary = enchant.Dict("en_US")
   
def remove_non_english(text):
        en_words= []
        for word in text:
            if dictionary.check(word):
                en_words.append(word)
        text = ' '.join(str(e) for e in en_words)
        return text

In [15]:
df['english_text'] = df['articles'].apply(lambda text: remove_non_english(text.split()))

In [16]:
df['english_text']

0       Statistical Machine Translation Draft of Chapt...
1       Found in Learning Robust Joint Representations...
2       Available online at Systems Engineering 5 427 ...
3       J OF I S AND E XX XXX-XXX Inverse-Category-Fre...
4       The University of T. E. Dunning Finding Struct...
                              ...                        
1059    Multi-label Hate Speech and Abusive Language D...
1060    The Universal Language Model in Arabic Hajj Am...
1061    An Online Arabic Sentiment School of Informati...
1062    2017 2nd International Conferences on Informat...
1063    INVESTIGATIVE RADIOLOGY Volume Number 473-481 ...
Name: english_text, Length: 1064, dtype: object

In [17]:
def finalpreprocess(string):
    return lemmatizer(stopword(preprocess(string)))


df['clean_text'] = df['english_text'].apply(lambda x: finalpreprocess(x))

In [18]:
df['clean_text']

0       statistical machine translation draft chapter ...
1       find learn robust joint representation cyclic ...
2       available online system engineering route opti...
3       xx xxx xxx inverse category frequency base sup...
4       university dun finding structure genome symbol...
                              ...                        
1059    multi label hate speech abusive language detec...
1060    universal language model arabic hajj american ...
1061    online arabic sentiment school informatics uni...
1062    nd international conference information inform...
1063    investigative radiology volume number company ...
Name: clean_text, Length: 1064, dtype: object

In [19]:
# count the number of words for each doc
df['clean_word_count'] = df['clean_text'].apply(lambda x: len(str(x).split()))
df["clean_word_count"]

0       18913
1        2581
2        1569
3        3124
4       30225
        ...  
1059     3246
1060     2447
1061     1704
1062      793
1063     2843
Name: clean_word_count, Length: 1064, dtype: int64

In [20]:
# total number of words of all articles after processing\n",
clean_count = [x for x in df['clean_word_count']]
    
total_count = 0
for i in count:
    total_count = total_count + i
    
total_count

8323220

In [21]:
# save the dataframe into file
df.to_csv('dataframe.csv')  

# Vectorization

In [22]:
#SPLITTING THE TEXTS FROM THE DATASET 
clean_text = df["clean_text"]

In [23]:
clean_text

0       statistical machine translation draft chapter ...
1       find learn robust joint representation cyclic ...
2       available online system engineering route opti...
3       xx xxx xxx inverse category frequency base sup...
4       university dun finding structure genome symbol...
                              ...                        
1059    multi label hate speech abusive language detec...
1060    universal language model arabic hajj american ...
1061    online arabic sentiment school informatics uni...
1062    nd international conference information inform...
1063    investigative radiology volume number company ...
Name: clean_text, Length: 1064, dtype: object

In [24]:
#Word2Vec
# Word2Vec runs on tokenized sentences
tok = [nltk.word_tokenize(i) for i in clean_text]  

In [25]:
tok[:2]

[['statistical',
  'machine',
  'translation',
  'draft',
  'chapter',
  'neural',
  'machine',
  'translation',
  'center',
  'speech',
  'language',
  'processing',
  'department',
  'computer',
  'science',
  'john',
  'hopkins',
  'university',
  'st',
  'public',
  'draft',
  'august',
  'nd',
  'public',
  'draft',
  'september',
  'content',
  'neural',
  'machine',
  'translation',
  'short',
  'history',
  'introduction',
  'neural',
  'network',
  'linear',
  'model',
  'multiple',
  'layer',
  'non',
  'linearity',
  'inference',
  'back',
  'propagation',
  'training',
  'computation',
  'graph',
  'neural',
  'network',
  'computation',
  'graph',
  'gradient',
  'computation',
  'deep',
  'learn',
  'framework',
  'neural',
  'language',
  'model',
  'fee',
  'forward',
  'neural',
  'language',
  'model',
  'word',
  'embed',
  'inference',
  'training',
  'recurrent',
  'neural',
  'language',
  'model',
  'long',
  'short',
  'term',
  'memory',
  'model',
  'gate',
  

In [26]:
model = Word2Vec(tok, min_count=1)
model

In [27]:
vector = model.wv['computer']  # get numpy vector of a word
sims = model.wv.most_similar('computer', topn=10)  # get other similar words
sims

[('flores', 0.7242020964622498),
 ('vision', 0.587537407875061),
 ('synthesis', 0.5577859878540039),
 ('rethink', 0.5510027408599854),
 ('electrical', 0.5121257901191711),
 ('robotics', 0.5081895589828491),
 ('aid', 0.5033887624740601),
 ('retrospective', 0.5001449584960938),
 ('audacity', 0.4964139461517334),
 ('chafe', 0.48712873458862305)]

In [28]:
# Store just the words + their trained embeddings.
word_vectors = model.wv
word_vectors.save("word2vec.wordvectors")

In [29]:
word_vectors.vectors

array([[-6.0183418e-01, -4.9008172e-02,  1.2340084e+00, ...,
        -3.4620291e-01,  1.2223936e+00,  2.7118641e-01],
       [-1.0510164e+00,  5.8151208e-02,  9.4172335e-01, ...,
        -1.7752095e-01,  5.7540011e-01,  2.5778502e-01],
       [-5.1805210e-01,  6.6115308e-01,  9.6763521e-01, ...,
         1.3266335e+00,  1.1876082e+00, -1.2548115e+00],
       ...,
       [ 9.7029703e-03,  2.3545269e-02, -6.4926674e-03, ...,
        -2.1200787e-02,  2.5700947e-02,  2.1905880e-03],
       [ 3.9714586e-04,  2.0153191e-02, -4.6266401e-03, ...,
        -2.1982422e-02,  1.7539162e-02,  1.3326762e-02],
       [ 2.6146719e-02, -1.6871073e-04, -3.6586441e-02, ...,
        -3.9279424e-02, -1.4389688e-02,  3.1703350e-03]], dtype=float32)

In [30]:
#building Word2Vec model
class MeanEmbeddingVectorizer(object):
    def __init__(self, word2vec):
        self.word2vec = word2vec
        # if a text is empty we should return a vector of zeros
        # with the same dimensionality as all the other vectors
        self.dim = len(next(iter(word2vec.values())))
        
    def fit(self, X, y):
        return self
    
    def transform(self, X):
        return np.array([
            np.mean([self.word2vec[w] for w in words if w in self.word2vec]
                    or [np.zeros(self.dim)], axis=0)
            for words in X
        ])
    


In [31]:
w2v = dict(zip(model.wv.index_to_key, model.wv.vectors))
modelw = MeanEmbeddingVectorizer(w2v)

# converting texts to numerical data using Word2Vec
vectors_w2v = modelw.transform(tok)

In [32]:
vectors_w2v

array([[ 0.23062752,  0.05846347,  0.14931606, ...,  0.2828695 ,
         0.7158993 ,  0.08272962],
       [ 0.12852754, -0.05002416,  0.2966123 , ...,  0.12639475,
         0.69486034,  0.42324898],
       [ 0.11443237,  0.10485353,  0.18440683, ..., -0.25590673,
         0.32264066,  0.3988514 ],
       ...,
       [-0.062307  ,  0.0659927 ,  0.18679771, ...,  0.11166867,
         0.3904316 ,  0.3736761 ],
       [ 0.11707183,  0.09293664,  0.34621787, ..., -0.07763323,
         0.35421494,  0.25474316],
       [ 0.28324857, -0.3079013 ,  0.4860787 , ..., -0.0533764 ,
         0.22241926,  0.6026357 ]], dtype=float32)

### Implementing LDA

In [33]:
!pip install pyLDAvis

/usr/lib/python3/dist-packages/secretstorage/dhcrypto.py:15: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/usr/lib/python3/dist-packages/secretstorage/util.py:19: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
Defaulting to user installation because normal site-packages is not writeable


In [34]:
#Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

#vis
import pyLDAvis
import pyLDAvis.gensim_models as gensimvis
pyLDAvis.enable_notebook()

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

In [35]:
id2word = corpora.Dictionary(tok)

corpus = []
for text in tok:
    new = id2word.doc2bow(text)
    corpus.append(new)

print (corpus[0][0:20])
# Create Dictionary
word = id2word[[0][:1][0]]
print (word)

[(0, 2), (1, 2), (2, 2), (3, 2), (4, 14), (5, 1), (6, 1), (7, 1), (8, 1), (9, 1), (10, 1), (11, 1), (12, 2), (13, 1), (14, 2), (15, 8), (16, 1), (17, 1), (18, 1), (19, 1)]
aaron


In [36]:
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=30,
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha="auto")

In [37]:
from pprint import pprint
# Print the Keyword in the 10 topics
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(3,
  '0.000*"recombine" + 0.000*"jerry" + 0.000*"thrice" + 0.000*"thirst" + '
  '0.000*"symptomatic" + 0.000*"subtend" + 0.000*"spectacularly" + 0.000*"so" '
  '+ 0.000*"allure" + 0.000*"montague"'),
 (7,
  '0.000*"recombine" + 0.000*"jerry" + 0.000*"thrice" + 0.000*"thirst" + '
  '0.000*"symptomatic" + 0.000*"subtend" + 0.000*"spectacularly" + 0.000*"so" '
  '+ 0.000*"allure" + 0.000*"montague"'),
 (17,
  '0.000*"recombine" + 0.000*"jerry" + 0.000*"thrice" + 0.000*"thirst" + '
  '0.000*"symptomatic" + 0.000*"subtend" + 0.000*"spectacularly" + 0.000*"so" '
  '+ 0.000*"allure" + 0.000*"montague"'),
 (8,
  '0.000*"recombine" + 0.000*"jerry" + 0.000*"thrice" + 0.000*"thirst" + '
  '0.000*"symptomatic" + 0.000*"subtend" + 0.000*"spectacularly" + 0.000*"so" '
  '+ 0.000*"allure" + 0.000*"montague"'),
 (5,
  '0.027*"simulates" + 0.000*"recombine" + 0.000*"jerry" + 0.000*"thrice" + '
  '0.000*"thirst" + 0.000*"symptomatic" + 0.000*"subtend" + '
  '0.000*"spectacularly" + 0.000*"so" + 0.000*

In [38]:
# Compute Perplexity
print('\nPerplexity: ', lda_model.log_perplexity(corpus))  # a measure of how good the model is. lower the better.



Perplexity:  -8.00589666732218


### Visulaize the data


In [39]:
pyLDAvis.enable_notebook()
vis = gensimvis.prepare(lda_model, corpus, id2word, mds="mmds", R=30)
vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
21    -0.126632 -0.104330       1        1  11.165452
23    -0.023147 -0.228119       2        1   8.106693
20     0.033837 -0.088418       3        1   7.974099
16    -0.190013 -0.012246       4        1   7.638620
15     0.062941 -0.189626       5        1   6.879027
29    -0.058513 -0.003822       6        1   6.085266
24     0.100512 -0.303063       7        1   6.075385
26    -0.049408 -0.353301       8        1   4.969343
27    -0.082082 -0.214570       9        1   4.644527
2     -0.068606 -0.291464      10        1   4.609523
6     -0.294133 -0.133360      11        1   4.524550
22    -0.304557 -0.058828      12        1   4.254322
25     0.118666 -0.110507      13        1   4.180778
18    -0.183756 -0.186276      14        1   3.964830
4     -0.191737  0.097285      15        1   3.638540
0     -0.255334  0.042037      16        1   2.898557
28    -0.355355  0.100037      17        1   2.511294
10    -0.286764 -0.303360      18        1   1.897781
14     0.288603 -0.382359      19        1   1.250483
19     0.245587 -0.179855      20        1   1.183243
1      0.336150 -0.160887      21        1   1.099588
13    -0.097827  0.414174      22        1   0.208668
9     -0.230351  0.469122      23        1   0.131252
11     0.087800  0.461513      24        1   0.064389
12     0.236678  0.393851      25        1   0.042402
5      0.262449  0.264458      26        1   0.000723
17     0.256248  0.265478      27        1   0.000168
8      0.256248  0.265478      28        1   0.000166
7      0.256248  0.265478      29        1   0.000166
3      0.256248  0.265478      30        1   0.000162, topic_info=           Term         Freq         Total Category  logprob  loglift
2150       word  32662.00000  32662.000000  Default  30.0000  30.0000
1050   language  19193.00000  19193.000000  Default  29.0000  29.0000
1225      model  59485.00000  59485.000000  Default  28.0000  28.0000
150   attention  11079.00000  11079.000000  Default  27.0000  27.0000
2064       user  11305.00000  11305.000000  Default  26.0000  26.0000
...         ...          ...           ...      ...      ...      ...
25       active      0.00025    704.780954  Topic30  -9.9166  -1.5221
26     activity      0.00025    286.268002  Topic30  -9.9166  -0.6211
27        actor      0.00025    438.598238  Topic30  -9.9166  -1.0478
28       actual      0.00025    515.436023  Topic30  -9.9166  -1.2092
29     actually      0.00025    408.186145  Topic30  -9.9166  -0.9759

[2099 rows x 6 columns], token_table=      Topic      Freq   Term
term                        
0         1  0.270584  aaron
0         5  0.030065  aaron
0         7  0.143643  aaron
0         9  0.314011  aaron
0        13  0.020043  aaron
...     ...       ...    ...
2167     16  0.024855   zero
2167     17  0.014794   zero
2167     19  0.260383   zero
5105      1  0.944284    zip
7358      7  0.816770    zit

[7384 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[22, 24, 21, 17, 16, 30, 25, 27, 28, 3, 7, 23, 26, 19, 5, 1, 29, 11, 15, 20, 2, 14, 10, 12, 13, 6, 18, 9, 8, 4])

/usr/lib/python3/dist-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
/usr/lib/python3/dist-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
/usr/lib/python3/dist-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
/usr/lib/python3/dist-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
/usr/lib/python3/dist-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alterna